In [1]:
import pandas as pd

In [2]:
octo = pd.read_csv('bad-scrape.raw.csv', names=[i for i in range(0, 4)])

In [3]:
octo_test = octo.copy()

problems (col name where attr resides for given prob in borked csv):
1. two empty columns separating name (0) and result (3)
2. one empty column preceding name (2) and result (3)

In [4]:
# remove indoor/outdoor rows
octo_test[0] = octo_test[0].str.rstrip()
octo_test.drop(list(octo_test.loc[octo_test[0].isin(['Indoor', 'Outdoor', 'Oudoor'])].index.values), inplace=True)

# where second col empty and fourth col populated, fill second col with contents of fourth (problem 1)
octo_test.loc[(octo_test[1].isnull()) & (octo_test[3].notnull()), 1] = octo_test.loc[octo_test[1].isnull()][3]

# where first column empty, fill it with value of second column (problem 2a)
octo_test.loc[(octo_test[0].isnull()), 0] = octo_test.loc[(octo_test[0].isnull())][1]

# where second column empty and third column populated, fill second col with contents of third (problem 2b)
octo_test.loc[(octo_test[1].isnull()) & (octo_test[2].notnull()), 1] = octo_test.loc[(octo_test[1].isnull()) & (octo_test[2].notnull())][2]

# remove dummy rows
octo_test.drop([2, 3], axis=1, inplace=True)

# reset the index
octo_test.reset_index(inplace=True, drop=True)

In [5]:
def process(df):
    # cleanup
    df.rename(columns={0: 'fixture_location', 1: 'result_value'}, inplace=True)
    df['park_name'] = ['' for i in range(0, len(df))]
    
    # get park names
    indices = df.loc[df.result_value.isnull()].index
    names = df.loc[df.result_value.isnull()].fixture_location
    parks = list(zip(indices, names))

    # restructure
    for ind, park in enumerate(parks):
        start, park = park
        try:
            end = parks[ind+1][0] - 1
        except IndexError:
            end = len(df)
        df.loc[start:end, 'park_name'] = park
        
    df.drop(list(indices), inplace=True)
    df['result_flag'] = df['result_value'].apply(lambda x: x.split(' ')[-1] if len(x.split(' ')) > 1 else '')
    df['result_value'] = df['result_value'].apply(lambda x: x.split(' ')[0]).str.replace('˂', '<')  

In [6]:
process(octo_test)
octo_test = octo_test.ix[:, [2, 0, 1, 3]]

In [7]:
octo_test.to_csv('chi-parks-lead-results-10-2016.processed.csv', index=False)